<a href="https://colab.research.google.com/github/SwordForShinobi/Retail-recommender-system/blob/main/Recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install git+https://github.com/benfred/implicit # я надеюсь проблема не в этом? Не в способе установки? Ведь все импорты потом ставятся
# !sudo -H python3 -m pip install implicit --no-cache --force-reinstall --log ./implicit.txt # разница есть! Если устанавливать так, то работает более-менее, но все равно хуже

# from implicit.als import AlternatingLeastSquares as ALS
# from implicit.nearest_neighbours import CosineRecommender as CR
# from scipy.sparse import csr_matrix as csr

import numpy as np
import pandas as pd

# Подготовка данных, минимальная разведка и т. п.

In [ ]:
trans = pd.read_csv('/content/drive/MyDrive/Skillbox/coursework_Recsys/transactions.csv')
trans.head()

order_id  user_id  order_number  ...  product_id  add_to_cart_order  reordered
0   2539329        1             1  ...         196                1.0        0.0
1   2539329        1             1  ...       14084                2.0        0.0
2   2539329        1             1  ...       12427                3.0        0.0
3   2539329        1             1  ...       26088                4.0        0.0
4   2539329        1             1  ...       26405                5.0        0.0

[5 rows x 9 columns]

In [ ]:
trans[(trans['product_id'] == 196) & (trans['user_id'] == 1)]

order_id  user_id  order_number  ...  product_id  add_to_cart_order  reordered
0    2539329        1             1  ...         196                1.0        0.0
5    2398795        1             2  ...         196                1.0        1.0
11    473747        1             3  ...         196                1.0        1.0
16   2254736        1             4  ...         196                1.0        1.0
21    431534        1             5  ...         196                1.0        1.0
29   3367565        1             6  ...         196                1.0        1.0
33    550135        1             7  ...         196                1.0        1.0
39   3108588        1             8  ...         196                2.0        1.0
47   2295261        1             9  ...         196                4.0        1.0
50   2550362        1            10  ...         196                1.0        1.0

[10 rows x 9 columns]

In [ ]:
trans.groupby(['user_id', 'product_id'])['add_to_cart_order'].sum().reset_index()

user_id  product_id  add_to_cart_order
0              1         196               14.0
1              1       10258               30.0
2              1       10326                5.0
3              1       12427               33.0
4              1       13032               19.0
...          ...         ...                ...
9459060   206209       43961               24.0
9459061   206209       44325                8.0
9459062   206209       48370                8.0
9459063   206209       48697                6.0
9459064   206209       48742               18.0

[9459065 rows x 3 columns]

In [ ]:
trans.groupby('user_id')['product_id'].value_counts() # В общем что, sum() считает правильно, но другой момент - а нужно ли это? Возможно нужно просто посчитать количество раз,
# когда добавляли продукт, без конкретного его числа в корзине.

user_id  product_id
1        196           10
         12427         10
         10258          9
         25133          8
         13032          3
                       ..
206209   40310          1
         41665          1
         44325          1
         48370          1
         48697          1
Name: product_id, Length: 9459065, dtype: int64

In [ ]:
trans.groupby(['user_id', 'product_id'])['add_to_cart_order'].size().reset_index() # то же самое, для использования в классах. Может так скоры станут требуемыми

user_id  product_id  add_to_cart_order
0              1         196                 10
1              1       10258                  9
2              1       10326                  1
3              1       12427                 10
4              1       13032                  3
...          ...         ...                ...
9459060   206209       43961                  3
9459061   206209       44325                  1
9459062   206209       48370                  1
9459063   206209       48697                  1
9459064   206209       48742                  2

[9459065 rows x 3 columns]

# Создание группировки

In [ ]:
grouped = trans.groupby(['order_id', 'user_id'])['product_id'].apply(lambda x: list(set(x))).reset_index() # группировка по заказам. Т е один и тот же юзер мог фигурировать в нескольких заказах и
# покупать разный контент
grouped.head()

order_id  user_id                                         product_id
0         3   205970  [17668, 24838, 17704, 46667, 21903, 17461, 326...
1         4   178520  [26434, 32645, 10054, 21351, 22598, 39758, 348...
2         5   156122  [48002, 45698, 18569, 37011, 15005, 8479, 9633...
3         7   142903                                     [46802, 34050]
4         8     3107                                            [23423]

In [ ]:
grouped['exploded_products'] = grouped.product_id.explode(ignore_index=True) # Тогда взрываем. Щас оперативка будет в шоке..

In [ ]:
example = grouped[grouped.user_id == 205970] # Это например все заказы для такого вот юзера и что он каждый раз покупал. Покупал надо сказать неплохо так, активно =)
# Теперь мы должны проанализировать этот частный случай, изучить заказы данного юзера и выдать рекомендацию. После чего по аналогии пройтись этим алгоритмом по всем юзерам.
# Это если в общих чертах. Тут 17704 продукт он очень любит =)

In [ ]:
products = pd.read_csv('/content/drive/MyDrive/Skillbox/coursework_Recsys/products.csv')
products.head()

product_id  ... department
0           1  ...     snacks
1           2  ...     pantry
2           3  ...  beverages
3           4  ...     frozen
4           5  ...     pantry

[5 rows x 6 columns]

In [ ]:
products[products.product_id == 17704] # Ага, вот что он любит =) packaged vegetables fruits

product_id product_name  ...                       aisle  department
17703       17704       Lemons  ...  packaged vegetables fruits     produce

[1 rows x 6 columns]

In [ ]:
grouped.info(), trans.info() # 26 млн строк в исх. датафрейме О_о

# Наивный алгоритм

In [ ]:
# хорошо, вот у нас есть наш екзампл, из предыдущего пункта:
example

In [ ]:
example.product_id[0]

[17668, 24838, 17704, 46667, 21903, 17461, 32665, 33754]

In [ ]:
items = example.product_id.explode().value_counts().reset_index() # создадим мини дф всего, что когда либо покупал этот юзер
items.columns = ['product_id', 'amount']
items.head(10) # вот например наш топ 10 для данного юзера
# И так можно со всеми, оборачиваем это в функции и т. п. 

product_id  amount
0       33754      17
1       21903      14
2       24838      14
3       17704      13
4       46667      13
5       24852       8
6       34255       7
7       16797       7
8        4957       7
9       49683       6

In [ ]:
sensei = pd.read_csv('/content/drive/MyDrive/Skillbox/coursework_Recsys/df_from_sensei.csv')

In [ ]:
groupa = trans.groupby(['user_id', 'product_id'])['add_to_cart_order'].size().reset_index() # да, size() дает вернее

In [ ]:
groupa[(groupa['user_id'] == 205970) & (groupa['product_id'] == 24852)]

user_id  product_id  add_to_cart_order
9448259   205970       24852                  8

In [ ]:
sensei[(sensei['user_id'] == 205970) & (sensei['product_id'] == 24852)]
# видим, что оба способа группировки идентичны

user_id  product_id  count
9448206   205970       24852      8

In [ ]:
# Сэнсэйская жесткая функция подсчета заказов:
from tqdm import tqdm
def make_counted_data(transactions, uniq_users):
    """
    функция подсчитывания заказов
    """
    counter_all_users = {}
    for user in tqdm(uniq_users):
        user_transactions = transactions[
            transactions["user_id"] == user
        ]  # выбор всех транзакций для определенного пользователя - user
        list_of_orders = user_transactions[
            "product_id"
        ]  # получения списка продуктов из всех транзакций пользователя
        order_counter = (
            list_of_orders.value_counts()
        )  # подсчет кол-ва заказов для каждого продукта
        order_counter_dict = order_counter.to_dict()
        counter_all_users[user] = order_counter_dict
    counted_arr = []

    # разложения словаря в более удобный формат
    for user in counter_all_users.keys():
        for prod, cnt in counter_all_users[user].items():
            counted_arr.append([user, prod, cnt])
    make_counted_df = pd.DataFrame(
        counted_arr, columns=["user_id", "product_id", "count"]
    )
    return make_counted_df

In [ ]:
pr = make_counted_data(trans, trans['user_id'].unique().tolist())
pr

100%|██████████| 100000/100000 [1:05:37<00:00, 25.40it/s]


user_id  product_id  count
0              1         196     10
1              1       12427     10
2              1       10258      9
3              1       25133      8
4              1       46149      3
...          ...         ...    ...
9459060   206209       20842      1
9459061   206209        2519      1
9459062   206209       23909      1
9459063   206209       26209      1
9459064   206209       14870      1

[9459065 rows x 3 columns]

In [ ]:
pr.to_csv('/content/drive/MyDrive/Skillbox/coursework_Recsys/df_from_sensei.csv', index=False)

In [ ]:
items.head(10).product_id.tolist()

[33754, 21903, 24838, 17704, 46667, 24852, 34255, 16797, 4957, 49683]

In [ ]:
sample_sub = pd.read_csv('/content/drive/MyDrive/Skillbox/coursework_Recsys/sample_submission.csv')
sample_sub.head()

user_id                                         product_id
0        1  196 19660 13176 5258 49235 40199 46149 37710 1...
1        2  24852 13176 16797 21137 21903 47766 45066 2784...
2        3  13176 24852 21137 21903 16797 47766 39275 2784...
3        7  13176 21137 24852 21903 27845 39275 16797 4923...
4       13  4210 24852 27086 13176 26209 47626 21137 47766...

In [ ]:
# собственно функция для нахождения топ10 для всех юзеров:
def naive_recommender(grouped):
  users_row = sample_sub.user_id.tolist()
  top10_row = []

  for i in sample_sub.user_id:
    example = grouped[grouped.user_id == i]
    items = example.product_id.explode().value_counts().reset_index()
    items.columns = ['product_id', 'amount']
    top10 = items.head(10).product_id.tolist()
    top10_row.append(top10)

  dataset = np.concatenate((np.array(users_row).reshape(100000, 1),
                            np.array(top10_row).reshape(100000, 1)), axis=-1)
  answer = pd.DataFrame(dataset, columns=['user_id', 'product_id'])
  return answer # чертовски долгий процесс

In [ ]:
preds = naive_recommender(grouped)
preds.head() # предсказываем 100к образцов, как в sample_submission (см функцию)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


users                                              items
0     1  [196, 12427, 10258, 25133, 46149, 13032, 26088...
1     2  [32792, 47209, 24852, 18523, 19156, 1559, 1658...
2     3  [39190, 47766, 21903, 17668, 9387, 43961, 1859...
3     7  [40852, 37602, 17638, 21137, 31683, 13198, 428...
4    13  [27086, 4210, 27435, 33735, 1689, 43086, 25134...

## Наивный алгоритм как класс

In [ ]:
import random

class Recommender:
  '''data must be DataFrame, others args(columns' names) - strings'''

  def __init__(self, data, order_column, user_column, product_column):# класс будет сам группировать исходный датафрейм для дальнейшей работы
      self.data = data
      self.order_column = order_column
      self.user_column = user_column
      self.product_column = product_column
      self.grouped = self.data.groupby([self.order_column, self.user_column])[self.product_column].apply(lambda x: list(set(x))).reset_index()

  def show_gpouped(self): # просто для просмотра группировки
    df = self.grouped
    return df
  
  def predict_user(self, user_number):
    '''input integer'''
    if len(self.grouped[self.grouped[self.user_column] == user_number][self.user_column]) > 0:
      example = self.grouped[self.grouped[self.user_column] == user_number]
      items = example.product_id.explode().value_counts().reset_index()
      items.columns = ['product_id', 'amount']

      return items.head(10)
    else:
      print('There is no such user =(')
    

  def predict_many_users(self, users_ids):
    '''users_ids - list of integers'''
    who = self.grouped.loc[self.grouped[self.user_column].isin(users_ids)]
    users_row = list(set(who[self.user_column]))
    top10_row = []

    for i in users_ids:
      example = self.grouped[self.grouped[self.user_column] == i]
      items = example.product_id.explode().value_counts().reset_index()
      items.columns = ['product_id', 'amount']
      top10 = items.head(10).product_id.tolist()
      top10_row.append(top10)

    answer = pd.DataFrame(users_row, columns=['user'])
    answer['top10'] = top10_row
    return answer

  def lol_prediction(self, users_ids, top_what=25):
    # Это гениальный метод, он формирует топ n товаров, (n получает на вход), и рандомом рекоммендует оттуда топ10. Сильно? =)))
    # Такой подход уже можно назвать рекоммендательной системой? Мои философские воззрения говорят, что можно. 
    '''users_ids - list of integers
        top_what - set of each user preferences, how many top prefs we take'''
    who = self.grouped.loc[self.grouped[self.user_column].isin(users_ids)]
    users_row = list(set(who[self.user_column]))
    top10_row = []

    for i in users_row:
      example = self.grouped[self.grouped[self.user_column] == i]
      items = example.product_id.explode().value_counts().reset_index()
      items.columns = ['product_id', 'amount']
      top_smth = items.head(top_what).product_id.tolist()
      top10 = random.choices(top_smth, k=10)
      top10_row.append(top10)

    answer = pd.DataFrame(users_row, columns=['user'])
    answer['top10'] = top10_row
    return answer
  

In [ ]:
rec = Recommender(trans, 'order_id', 'user_id', 'product_id')

In [ ]:
rec.show_gpouped().head()

In [ ]:
top_10 = rec.predict_many_users(trans['user_id'].unique().tolist())
top_10_list = top_10['top10'].tolist()
np.save('/content/drive/MyDrive/Skillbox/coursework_Recsys/top_10_list.npy', top_10_list)
# top_10.to_csv('/content/drive/MyDrive/Skillbox/coursework_Recsys/top_10_df.csv', index=False)

In [ ]:
rec.predict_user(1)


product_id  amount
0         196      10
1       12427      10
2       10258       9
3       25133       8
4       46149       3
5       13032       3
6       26088       2
7       26405       2
8       49235       2
9       13176       2

In [ ]:
rec.lol_prediction(trans['user_id'].unique().tolist(), 10)

KeyboardInterrupt: ignored

# Предсказания, заключит. часть (для отправки на валидацию)

In [ ]:
preds = rec.lol_prediction(trans['user_id'].unique().tolist(), 50)
preds

user                                              top10
0           1  [10326, 10258, 13032, 13176, 38928, 41787, 171...
1           2  [17758, 17872, 47209, 5907, 32052, 47766, 4821...
2           3  [8021, 42557, 28373, 1005, 12845, 18599, 1005,...
3           7  [42265, 14332, 9598, 39121, 33740, 519, 19678,...
4          13  [42248, 32850, 1689, 18168, 18168, 42248, 4148...
...       ...                                                ...
99995  206202  [21137, 35640, 18567, 49191, 46642, 19353, 392...
99996  206206  [40124, 16387, 11520, 5876, 25332, 48679, 2900...
99997  206207  [33716, 30233, 1360, 39040, 33754, 3397, 46667...
99998  206208  [9339, 11520, 27845, 27744, 23198, 27690, 1877...
99999  206209  [6567, 4591, 41665, 41665, 28590, 22920, 36056...

[100000 rows x 2 columns]

In [ ]:
preds.columns = ['user_id', 'product_id']

In [ ]:
# Убираем лишние символы из столбца items для создания итогового csv файла для загрузки на kaggle
characters_to_remove = '[],' # пунктуационный фильтр
clear_items_list = []

for string in preds.product_id:
  for i in characters_to_remove:
    string = string.replace(i, '')
  clear_items_list.append(string)

preds.product_id = clear_items_list
preds.head()

user_id                                         product_id
0        1  10326 10258 13032 13176 38928 41787 17122 1303...
1        2  17758 17872 47209 5907 32052 47766 48210 35917...
2        3  8021 42557 28373 1005 12845 18599 1005 22035 1...
3        7  42265 14332 9598 39121 33740 519 19678 29894 2...
4       13  42248 32850 1689 18168 18168 42248 41480 5652 ...

In [ ]:
%cd /content/drive/MyDrive/Skillbox/coursework_Recsys

preds.to_csv('my_submission_random_choice.csv', index=False, encoding='utf-8', sep=',')

/content/drive/MyDrive/Skillbox/coursework_Recsys


# Вслед за сэнсэем

## Агрументы сэнсэя

In [ ]:
max(np.unique(dw.trans_df["user_id"].values))

In [ ]:
######## Ищу ошибки
dw = PrepareData()
dw.read_transactions("/content/drive/MyDrive/skillbox/transactions.csv")
dw.read_products("/content/drive/MyDrive/skillbox/products.csv")
dw.create_user_list()
dw.save_user_list("/content/users_uniq.npy")
dw.each_user_orders('user_id', 'product_id', 'add_to_cart_order') # user -1 , product_id - 196 , add_to_cart_order - 14
###########

Saved successfully!


user_id  product_id  add_to_cart_order
0              1         196                 10
1              1       10258                  9
2              1       10326                  1
3              1       12427                 10
4              1       13032                  3
...          ...         ...                ...
9459060   206209       43961                  3
9459061   206209       44325                  1
9459062   206209       48370                  1
9459063   206209       48697                  1
9459064   206209       48742                  2

[9459065 rows x 3 columns]

In [ ]:
x = dw.trans_df[dw.trans_df["user_id"] == 1] #12669,24852,18
x[x["product_id"] == 196] # user -1 , product_id - 196 , add_to_cart_order - 10 получается ошибочка
# Я вот 2 раза посчитал add_to_cart_order, даже вслух - там получается 14 - было правильно

order_id  user_id  order_number  ...  product_id  add_to_cart_order  reordered
0    2539329        1             1  ...         196                1.0        0.0
5    2398795        1             2  ...         196                1.0        1.0
11    473747        1             3  ...         196                1.0        1.0
16   2254736        1             4  ...         196                1.0        1.0
21    431534        1             5  ...         196                1.0        1.0
29   3367565        1             6  ...         196                1.0        1.0
33    550135        1             7  ...         196                1.0        1.0
39   3108588        1             8  ...         196                2.0        1.0
47   2295261        1             9  ...         196                4.0        1.0
50   2550362        1            10  ...         196                1.0        1.0

[10 rows x 9 columns]

In [ ]:
dw.trans_df.groupby(["user_id", 'product_id'])['add_to_cart_order'].sum().reset_index()

user_id  product_id  add_to_cart_order
0              1         196               14.0
1              1       10258               30.0
2              1       10326                5.0
3              1       12427               33.0
4              1       13032               19.0
...          ...         ...                ...
9459060   206209       43961               24.0
9459061   206209       44325                8.0
9459062   206209       48370                8.0
9459063   206209       48697                6.0
9459064   206209       48742               18.0

[9459065 rows x 3 columns]

In [ ]:
df = dw.to_fit_coded()

In [ ]:
df.iloc[:2]

user_id  product_id  ...  product_id(coded)  number_of_orders(coded)
0        1         196  ...                195                        9
1        1       10258  ...              10216                        8

[2 rows x 6 columns]

In [ ]:
recommendations = [195,10216]
df.loc[recommendations][df.columns[1]].values.tolist() # кусок из predict_many декодирует 154 в 519, хотя на деле 154 должен декодироваться в 196


[30391, 42637]

In [ ]:
df[df["product_id(coded)"].isin(recommendations)][df.columns[1]].unique()

array([  196, 10258])

## Подготовка данных (класс)

Создадим класс с некст методами:

1) чтение файлов transactions и products

2) создание списка уникальных пользователей 

3) сохранение списка уникальных пользователей

4) чтение уникальных пользователей из файла

5) добавить строку в transactions

6) добавить строку в products

7) создание набора данных со следующими столбцами - [ "user_id" , "product_id" , "number_of_orders"] , где  "number_of_orders"  - сколько раз данный юзер заказал данный продукт

8) сохранение данных из п.7

9) чтение данных из п.8

In [ ]:
class PrepareData:
  '''IMPORTANT! Use .read transactions() and/or .read_products() before, otherwise other methods will be unavaliable'''
  def __init__(self):
    self.trans_df = None
    self.prods_df = None
    self.purchases = None

  def read_transactions(self, trans_dir=None): # чтение файлов transactions и products
    '''Pass directory(string) of desired file you wish to create DataFrame from'''
    if trans_dir is not None:
      self.trans_df = pd.read_csv(trans_dir, sep=',')
      return self.trans_df
    else:
      print('Please, input directory')

  def read_products(self, prods_dir=None):
    '''Pass directory(string) of desired file you wish to create DataFrame from'''
    if prods_dir is not None:
      self.prods_df = pd.read_csv(prods_dir, sep=',')
      return self.prods_df
    else:
      print('Please, input directory')

  def create_user_list(self): # уникальные user_id из transactions
    if self.trans_df is not None:
      self.user_list_trans = self.trans_df.user_id.unique().tolist()
      return self.user_list_trans
    else:
      raise ValueError('Load data 1st! Read some dataframes!')

  def save_user_list(self, directory):
    '''type full directory, must be a string'''
    if self.user_list_trans is None:
        self.create_user_list()
    np.save(directory, self.user_list_trans)
    print("Saved successfully!")

  def read_user_list(self, directory): # чтение из файла списка уникальных юзеров
    '''type full directory, must be a string'''
    self.user_list_trans = np.load(directory+'.npy', allow_pickle=True)
    return list(self.user_list_trans)

  def trans_add_string(self, order_id, user_id, order_number, order_dow,
                       order_hour_of_day, days_since_prior_order, product_id,
                       add_to_cart_order, reordered): # + новая строка в транзакции
    empty_row = pd.DataFrame(np.array([order_id, user_id, order_number, order_dow, order_hour_of_day,
                              days_since_prior_order, product_id, add_to_cart_order, reordered]).reshape(1, len(self.trans_df.columns)),
                             columns=list(self.trans_df.columns))
    new_trans_df = self.trans_df.append(empty_row, ignore_index=True)
    return new_trans_df

  def prods_add_string(self, product_id, product_name, aisle_id, department_id,
                       aisle, department): # + новая строка в продукты
    empty_row = pd.DataFrame(np.array([product_id, product_name, aisle_id, department_id,
                              aisle, department]).reshape(1, len(self.prods_df.columns)),
                             columns=list(self.prods_df.columns))
    new_prods_df = self.prods_df.append(empty_row, ignore_index=True)
    return new_prods_df

  def each_user_orders(self, users, products, number_of_orders): # сколько раз отдельный юзер покупал каждый продукт
    '''all are strings:
        users - name of columns with user ids,
        products - name of columns with product ids,
        number_of_orders - counts purchases of each product ever done by the user'''
    self.users = users
    self.products = products
    self.number_of_orders = number_of_orders

    self.purchases = self.trans_df.groupby([users, products])[number_of_orders].size().reset_index()
    return self.purchases

  def save_user_orders(self, directory): # сохранение посчитанных покупок каждого юзера
    if self.purchases is not None:
      self.purchases.to_csv(directory, index=False, sep=',', encoding='utf-8')
      print("Saved successfully!")
    else:
      raise ValueError('Nothing to save! Create dataframe 1st!')

  def read_user_orders(self, directory): # загрузка посчитанных покупок каждого юзера (ранее сохраненных)
    self.purchases = pd.read_csv(directory, sep=',')
    return self.purchases
  
  def to_fit_coded(self):
    if self.purchases is not None:
      df = self.purchases
      for i in [self.users, self.products, self.number_of_orders]:
        df[i+'_coded'] = df[i]
      for i in [self.users+'_coded', self.products+'_coded', self.number_of_orders+'_coded']:
        df[i] = df[i].astype("category")
        df[i] = df[i].cat.codes
      df.columns = ['user_id', 'product_id', 'number_of_orders',
                    'user_id(coded)', 'product_id(coded)', 'number_of_orders(coded)']
      return df
    else:
      raise ValueError('Use each_user_orders() method 1st')
  
  def to_fit_pure(self): # так ли нам нужно кодировать? Ведь потом придется РАСкодировтаь +)
    if self.purchases is not None:
      df = self.purchases
      df.columns = ['user_id', 'product_id', 'number_of_orders']
      return df
    else:
      raise ValueError('Use each_user_orders() method 1st')

In [ ]:
data = PrepareData()

In [ ]:
data.read_transactions('/content/drive/MyDrive/Skillbox/coursework_Recsys/transactions.csv')
data.read_products('/content/drive/MyDrive/Skillbox/coursework_Recsys/products.csv')
data.each_user_orders('user_id', 'product_id', 'add_to_cart_order')

user_id  product_id  add_to_cart_order
0              1         196                 10
1              1       10258                  9
2              1       10326                  1
3              1       12427                 10
4              1       13032                  3
...          ...         ...                ...
9459060   206209       43961                  3
9459061   206209       44325                  1
9459062   206209       48370                  1
9459063   206209       48697                  1
9459064   206209       48742                  2

[9459065 rows x 3 columns]

## Модель (ALS)

In [ ]:
# Создадим учебный cет КОДИРОВАННЫЙ + столбцы с истинными значениями, чтобы потом в итоге формировать расшифрованные предсказания для валидации
to_fit = data.to_fit_coded()
to_fit.head()

user_id  product_id  ...  product_id(coded)  number_of_orders(coded)
0        1         196  ...                195                        9
1        1       10258  ...              10216                        8
2        1       10326  ...              10282                        0
3        1       12427  ...              12375                        9
4        1       13032  ...              12976                        2

[5 rows x 6 columns]

In [ ]:
import pickle
class ModelALS:

  def __init__(self, factors=None, iterations=None):
    self.factors = factors
    self.iterations = iterations

    if self.factors is not None and self.iterations is not None:
      self.model = ALS(self.factors, self.iterations)
    else:
      raise ValueError('Please set parameters when create class')

  def prepare_matrix_to_fit(self, df=None):
    self.df = df
    if self.df is not None: # передаем сюда ЗАКОДИРОВАННЫЕ столбцы
      row = np.array(self.df[self.df.columns[3]].values.tolist()) # users
      col = np.array(self.df[self.df.columns[4]].values.tolist()) # products
      data = np.array(self.df[self.df.columns[5]].values.tolist()) # how many purchased
      self.matrix = csr((data, (row, col)), dtype=np.float32)
      return self.matrix
    else:
      raise ValueError('Please pass dataframe to work with')
  
  def fit_model(self):
    self.model.fit(self.matrix.T)

  def predict_user(self, user, n, filter_already_liked_items=True):
    '''user = particular user id(integer)
        n = amount of items to predict(integer)'''
    code_user = self.df[self.df[self.df.columns[0]] == user][self.df.columns[3]].unique()[0]
    recommendations = [t[0] for t in self.model.recommend(code_user, self.matrix, n, filter_already_liked_items)]
    recs = self.df[self.df[self.df.columns[4]].isin(recommendations)][self.df.columns[1]].unique().tolist()
    return f'For user_id={user}, we recommend next items, ids: {recs}'

  def predict_many(self, users, n, filter_already_liked_items=True):
    '''users = users you wish to predict(list)
        n = amount of items to predict for each user(integer)'''
    self.users = users
    self.preds_list = []
    users_code = self.df[self.df[self.df.columns[0]].isin(users)][self.df.columns[3]].unique().tolist()
    decode_df = self.df[[self.df.columns[4], self.df.columns[1]]].drop_duplicates() # создадим короткий дф для раскодировки. Ниже эксперементально проверенно:
    # он очень сильно ускоряет процесс
    for i in users_code:
      recommendations = [t[0] for t in self.model.recommend(i, self.matrix, n, filter_already_liked_items, recalculate_user=True)]
      self.preds_list.append(recommendations)
    return self.preds_list

  def decode_predictions(self, preds_list = None):
    self.preds_list = preds_list
    if self.preds_list is not None:
      decode_df = self.df[[self.df.columns[4], self.df.columns[1]]].drop_duplicates() # создадим короткий дф для раскодировки. Ниже экспериментально проверенно
      decode = []

      for i in self.preds_list:
        decoded_string = decode_df[decode_df['product_id(coded)'].isin(i)]['product_id'].unique().tolist()
        decode.append(decoded_string)
      # Сразу уберем пунктуацию для сохр-я и отправки на кэггл:
      characters_to_remove = '[],'
      clear_items_list = []

      for string in decode:
        for i in characters_to_remove:
          string = str(string).replace(i, '')
        clear_items_list.append(string)
  
      answer = pd.DataFrame(np.array(self.users).reshape(len(self.users), 1), columns=['user_id'])
      answer['product_id'] = clear_items_list
      return answer
    else:
      raise ValueError('Use predict_many() method 1st then pass predictions')

  def save_model(self, path=None):
    if path is not None:
      with open(path, 'wb') as directory:
        pickle.dump(self.model, directory)
    else:
      raise ValueError('Please input desirable path/filename to save to')
    
  def load_model(self, path=None):
    if path is not None:
      with open(path, 'rb') as model_probe:
        self.model = pickle.load(model_probe)
    else:
      raise ValueError('Please input desirable path/filename to load from')

In [ ]:
model = ModelALS(30, 8)

In [ ]:
matrix = model.prepare_matrix_to_fit(to_fit)


In [ ]:
model.fit_model()

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
model.save_model('/content/drive/MyDrive/Skillbox/coursework_Recsys/Saved_models/probe(30_8)')

In [ ]:
model.load_model('/content/drive/MyDrive/Skillbox/coursework_Recsys/Saved_models/probe(30_8)')

In [ ]:
model.predict_user(2653, 10)

'For user_id=2653, we recommend next items, ids: [13176, 7781, 24852, 27966, 21137, 43961, 26209, 9076, 47626, 38739]'

In [ ]:
preds = model.predict_many(to_fit['user_id'].unique().tolist(), 10)

In [ ]:
to_validation = model.decode_predictions(preds)
to_validation.to_csv('/content/drive/MyDrive/Skillbox/coursework_Recsys/submission_modelALS(30_8_decoding).csv', index=False)

## kNN method

In [ ]:
class kNN(ModelALS): # наследуемся, чтобы не переписывать функции заново
  def __init__(self, k=50):
    self.k=k
    self.model = CR(self.k)

  def prepare_matrix_to_fit(self, df=None):
    self.df = df
    if self.df is not None:
      row = np.array(self.df[self.df.columns[3]].values.tolist()) # users
      col = np.array(self.df[self.df.columns[4]].values.tolist()) # products
      data = np.array(self.df[self.df.columns[5]].values.tolist()) # how many purchased
      self.matrix = csr((data, (row, col)), dtype=np.double) # ради этого пришлось писать функцию полностью
      return self.matrix
    else:
      raise ValueError('Please pass dataframe to work with')

In [ ]:
model_knn = kNN(10)

In [ ]:
matrix_knn = model_knn.prepare_matrix_to_fit(to_fit)

In [ ]:
model_knn.fit_model()

/usr/local/lib/python3.7/dist-packages/implicit/nearest_neighbours.py:184: RuntimeWarning: invalid value encountered in true_divide
  X.data = X.data / sqrt(bincount(X.row, X.data ** 2))[X.row]


  0%|          | 0/49465 [00:00<?, ?it/s]

In [ ]:
preds = model_knn.predict_many(to_fit['user_id'].unique().tolist(), 10)


In [ ]:
to_validation = model_knn.decode_predictions(preds)
to_validation.to_csv('/content/drive/MyDrive/Skillbox/coursework_Recsys/submission_kNN(10_coded_1101_1).csv', index=False)

In [ ]:
model_knn.save_model('/content/drive/MyDrive/Skillbox/coursework_Recsys/Saved_models/probeKNN(10)')

In [ ]:
model_knn.load_model('/content/drive/MyDrive/Skillbox/coursework_Recsys/Saved_models/probeKNN(10)')

## Проверки, тренировки, гипотезы

In [ ]:
row = to_fit['user_id(coded)'].values
col = to_fit['product_id(coded)'].values
data = to_fit['number_of_orders'].values
matrix_user_item = csr((data, (row, col)), dtype=np.double)
matrix_user_item.shape

(100000, 49465)

In [ ]:
model_knn = CR(10)
model_knn.fit(matrix_user_item.T) # учится без сообщения, интересно. И наверно, хорошо

  0%|          | 0/49465 [00:00<?, ?it/s]

In [ ]:
# тут не трогал, не до этого
def show_recommendations(model, user, n, df=to_fit):
  code_user = df[df[df.columns[0]].isin([user])][df.columns[3]].unique()[0]
  recommendations = [t[0] for t in model.recommend(code_user, matrix_user_item, n)]
  recs = df[df[df.columns[4]].isin(recommendations)][df.columns[1]].unique().tolist()
  return f'For user id #{user} we recommend next items id: {recs}'

In [ ]:
# ЗАКОДИРОВАННЫЕ предсказания:
def predict_many(model, users, n, filter_already_liked_items=True, df=to_fit):

  code_users = df.loc[df[df.columns[0]].isin(users)][df.columns[3]].unique().tolist()
  preds_list = []

  for i in code_users:
    recommendations = [t[0] for t in model.recommend(i, matrix_user_item, n, filter_already_liked_items, recalculate_user=True)]
    preds_list.append(recommendations)

  return preds_list

In [ ]:
# Как оно предсказывало раньше: (Пока без рекалкулэйта и прочего)
recos = model_knn.recommend(0, matrix_user_item, 10) # здесь вылетает колаб (перезапускается). Но не вылетает, если устанавливать имплисит по-старому
recos

[(6155, 8.712250746615215),
 (37552, 8.623389566354215),
 (31515, 5.952280145121511),
 (41223, 3.7148694630849333),
 (13366, 3.0399796191582404),
 (13517, 2.8577196144777703),
 (45863, 2.754092746533307),
 (42317, 2.7405876650038126),
 (12986, 2.394782538174308),
 (4449, 2.192013140265999)]

In [ ]:
recos = [t[0] for t in  model_knn.recommend(0, matrix_user_item, 10)] # если импортировать по-старому, то и предсказания работают по-старому
recos

[6155, 37552, 31515, 41223, 13366, 13517, 45863, 42317, 12986, 4449]

In [ ]:
probe = predict_many(model_knn, to_fit['user_id'].unique().tolist(), 10)

In [ ]:
decode_df = to_fit[['product_id(coded)', 'product_id']].drop_duplicates()

In [ ]:
decode = []
for i in probe:
  decoded_string = decode_df[decode_df['product_id(coded)'].isin(i)]['product_id'].unique().tolist()
  decode.append(decoded_string)

In [ ]:
characters_to_remove = '[],' # пунктуационный фильтр
clear_items_list = []

for string in decode:
  for i in characters_to_remove:
    string = str(string).replace(i, '')
  clear_items_list.append(string)
ans = pd.DataFrame(to_fit['user_id'].unique().reshape(100000, 1), columns=['user_id'])
ans['product_id'] = clear_items_list

In [ ]:
ans.to_csv('/content/drive/MyDrive/Skillbox/coursework_Recsys/probe_1101_2.csv', index=False)

In [ ]:
recs = show_recommendations(model_knn, user=77, n=10) # топ10 для 77 юзера, например
recs

'For user id #77 we recommend next items id: [13176, 17758, 27966, 47766, 49683, 26209, 39275, 49247, 11512, 37788]'

In [ ]:
with open(r'/content/drive/MyDrive/Skillbox/coursework_Recsys/Saved_models/probe', 'wb') as path:
  pickle.dump(model_knn, path)

In [ ]:
with open(r'/content/drive/MyDrive/Skillbox/coursework_Recsys/Saved_models/probe', 'rb') as model_probe:
  model_als = pickle.load(model_probe)